In [4]:
import os
import pandas as pd
import numpy as np
import pickle
import collections
from Graham.Aggregation import Aggregate
from Graham.Standard import Standardize
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.width', 1500)
pd.set_option('display.precision', 2)
pd.options.display.float_format = '{:20,.2f}'.format

In [8]:
symbols = np.array(os.listdir('/Users/duncangh/PycharmProjects/FSA/data/financials_data/')[1:])

In [11]:
stock = Standardize('ADS')
income = stock.income
cash = stock.cash
balance = stock.balance

In [21]:
def aggregate(symbols):
    stock = Standardize('ADS')
    main_income = stock.income
    main_cash = stock.cash
    main_balance = stock.balance
    
    for sym in symbols:
        try:
            sub_stock = Standardize(sym)
            main_income.append(sub_stock.income)
            main_cash.append(sub_stock.cash)
            main_balance.append(sub_stock.balance)
        except:
            pass
    
    return main_income, main_cash, main_balance
main_income, main_cash, main_balance = aggregate(symbols[1:])

In [ ]:
stock = Standardize('ADS')
income = []
for sym in symbols:
    try:
        income = Standardize(sym).income
        income.loc[['Revenues', 'OperatingExpenses', 'OperatingIncomeLoss']]
    except:
        pass
    

In [52]:
income = Standardize(sym).income
fin = income.loc[['Revenues', 'OperatingExpenses', 'OperatingIncomeLoss']]

In [55]:
fin.index = pd.MultiIndex.from_product([fin.index.values, [sym]])
fin

,,2009,2010,2011,2012,2013,2014,2015
Revenues,TSLA,111.94,116.74,204.24,413.26,"2,013.50","3,198.36","4,046.03"
OperatingExpenses,TSLA,61.43,177.57,313.08,424.35,517.54,"1,068.36","1,640.13"
OperatingIncomeLoss,TSLA,-51.90,-146.84,-251.49,-394.28,-61.28,-186.69,-716.63


In [57]:
i2 = Standardize('AFL').income.loc[['Revenues', 'OperatingExpenses', 'OperatingIncomeLoss']]
i2.index = pd.MultiIndex.from_product([i2.index.values, ['AFL']])
i2

,,2007,2008,2009,2010,2011,2012,2013,2014,2015
Revenues,AFL,"15,393.00","16,554.00","18,254.00","20,732.00","22,171.00","25,364.00","23,939.00","22,728.00","20,872.00"
OperatingExpenses,AFL,nan,nan,nan,nan,nan,nan,nan,nan,nan
OperatingIncomeLoss,AFL,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [60]:
pd.concat([i2, fin])

,,2007,2008,2009,2010,2011,2012,2013,2014,2015
Revenues,AFL,"15,393.00","16,554.00","18,254.00","20,732.00","22,171.00","25,364.00","23,939.00","22,728.00","20,872.00"
OperatingExpenses,AFL,nan,nan,nan,nan,nan,nan,nan,nan,nan
OperatingIncomeLoss,AFL,nan,nan,nan,nan,nan,nan,nan,nan,nan
Revenues,TSLA,nan,nan,111.94,116.74,204.24,413.26,"2,013.50","3,198.36","4,046.03"
OperatingExpenses,TSLA,nan,nan,61.43,177.57,313.08,424.35,517.54,"1,068.36","1,640.13"
OperatingIncomeLoss,TSLA,nan,nan,-51.90,-146.84,-251.49,-394.28,-61.28,-186.69,-716.63
